In [13]:
from os import sys
sys.path.append("/meshAfterParty")

# Future Datajoint Pipeline

In [ ]:
"""
All neurons
1) Start: Decimated Meshes
2) Preprocess all meshes into neron object
3) Begin to fill out DataJoint Table


Datajoint table:
Key: segment_id

Attributes: (WHERE THE NEURON PACKAGE FITS IN)
- spine count
- axon_traveling_description
- axon locations (np.array    N x 3)
- soma center ( 1 x 3 array)



Process: Make a populate function that
For each preprocessed neuron
a. Compute the attributes
b. Store Attribute in datajoint table
Finshed product: Populated datajoint table

Analysis: Query the datajoint table:

soma_center < [layer_6_y_value] & axon.....
--> Returns: list of martinoti cell segment ids

"""

"""
Populate your table on the cluster

"""



# Importing a Neuron to Process

In [21]:
import trimesh_utils as tu
inhibitory_neuron = tu.load_mesh_no_processing("/notebooks/test_neurons/spine_detection/90725377802114822_inhibitory_7.off")
double_soma_neuron = tu.load_mesh_no_processing("/notebooks/test_neurons/multi_soma_example.off")


TypeError: 'Trimesh' object is not callable

# Pre-Processing the Neurons

In [ ]:
import neuron
import time

start_time = time.time()
inhibitory_obj = neuron.Neuron(inhibitory_neuron,
                              segment_id = 90725377802114822,
                              description="inhibitory_7",
                              decomposition_type="meshafterparty")
print(f"Total time for processing inhibitory neuron = {time.time() - start_time}")

--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....


In [ ]:
import neuron
import time

start_time = time.time()
double_neuron_obj = neuron.Neuron(double_soma_neuron,
                              segment_id = 12345,
                              description="double_soma",
                              preprocessing_decomposition_type="meshparty")
print(f"Total time for processing inhibitory neuron = {time.time() - start_time}")

# Visualizing Neurons

In [ ]:
import skeleton_utils as sk
sk.graph_skeleton_and_mesh(other_meshes=[inhibitory_obj.mesh])

In [ ]:
inhibitory_obj.plot_soma_limb_concept_network()

In [ ]:
import neuron_visualizations as nviz

In [ ]:
nviz.visualize_neuron(inhibitory_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all",
                     mesh_color="green",
                     skeleton_color="black")

In [ ]:
nviz.visualize_neuron(inhibitory_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all",
                     mesh_color="random",
                     skeleton_color="random")

In [ ]:
nviz.visualize_neuron(inhibitory_obj,
                     visualize_type=["network","mesh"],
                     limb_branch_dict="all",
                      mesh_limb_branch_dict="all",
                      mesh_soma=True,
                      
                     node_size=0.8,
                     arrow_size=0.8)

# Calculating Spines

In [ ]:
inhibitory_obj.calculate_spines()

In [ ]:
nviz.plot_spines(inhibitory_obj)

# Neuron Object

In [ ]:
dir(inhibitory_obj)

In [ ]:
inhibitory_obj.concept_network

In [ ]:
# networkx -> a way of storing graph stuff 
import networkx as nx
#inhibitory_obj.plot_soma_limb_concept_network()
nx.draw(inhibitory_obj.concept_network,with_labels=True)

# Soma Object

In [ ]:
soma_object = inhibitory_obj.concept_network.nodes["S0"]["data"]
#dir(soma_object)

In [ ]:
# What verticies make up the face, index of the verticies.
soma_object.mesh.faces

In [ ]:
soma_object.mesh.vertices

In [ ]:
sk.graph_skeleton_and_mesh(other_meshes=[soma_object.mesh])

__init__

Neuron()

__eq__
x == Y

__getitem__(index= 4)

x[4]

**kwargs

# Limb Object

In [ ]:
limb_idx = "L0"
limb_obj = inhibitory_obj.concept_network.nodes[limb_idx]["data"]

nviz.visualize_neuron(inhibitory_obj,
                      visualize_type=["mesh","skeleton"],
                    limb_branch_dict=dict(L0=["all"]),
                      mesh_whole_neuron=True,
                      mesh_color_alpha=0.1
                     )

In [ ]:
dir(limb_obj)

In [ ]:
nviz.visualize_neuron(inhibitory_obj,
                     limb_branch_dict=dict(L0=[7]),
                     visualize_type=["mesh","skeleton"],
                      mesh_color="red",
                      mesh_color_alpha=1,
                     mesh_whole_neuron=True)

In [ ]:
limb_obj.concept_network.nodes

# Branch Object

In [ ]:
branch_obj = limb_obj.concept_network.nodes[7]["data"]

In [ ]:
dir(branch_obj)

In [ ]:
branch_obj.spines

In [ ]:
limb_obj.concept_network.nodes()

# How to Query a Neuron object

In [ ]:
# limb_obj.concept_network.nodes[4]["data"].labels.append("Axon")
# limb_obj.concept_network.nodes[1]["data"].labels.append("Axon")
# limb_obj.concept_network.nodes[2]["data"].labels.append("Axon")

In [8]:
def new_function(x,y,**kwargs):
    print(kwargs)
    return x + y

arg_var = dict(dc_voltage=6, b = 4)
new_function(4,5,**arg_var)

{'dc_voltage': 6, 'b': 4}


9

In [ ]:
from neuron_searching import run_options
# Define some function list
# You'll receive either a limb or a branch, and you'll need to do something about it
# Use the file -> meshafterparty\neuron_searching.py
@run_options(run_type="Limb")
def limb_n_spine(curr_limb,limb_name=None,**kwargs):
    """
    Pseudocode: 
    1) Get all the branch names in the limb
    2) For each branch:
        a. get the number of spines and add to total
    3) Return total spine count
    """
    
    #1) Get all the branch names in the limb
    branch_names = list(curr_limb.concept_network.nodes())
     
    #2) For each branch:
    #a. get the number of spines and add to total
    total_spines = 0
    for b in branch_names:
        branch_obj = curr_limb.concept_network.nodes[b]["data"]
        
        if branch_obj.spines is None:
            continue
        else:
            total_spines += len(branch_obj.spines)
        
    #3) Return total spine count
    return total_spines

@run_options(run_type="Limb")
def limb_n_spine_per_skeletal_length(curr_limb,
                                     limb_name=None,
                                     **kwargs):
    """
    Pseudocode: 
    1) Get all the branch names in the limb
    2) For each branch:
        a. get the number of spines and add to total
    3) Get total skeletal length
    4) Return n_spine/length
    """
    
    #1) Get all the branch names in the limb
    branch_names = list(curr_limb.concept_network.nodes())
    
    #2) For each branch:
    #a. get the number of spines and add to total
    total_spines = 0
    for b in branch_names:
        branch_obj = curr_limb.concept_network.nodes[b]["data"]
        
        if branch_obj.spines is None:
            continue
        else:
            total_spines += len(branch_obj.spines)
        
    #3) Get total skeletal length
    import skeleton_utils as sk
    skeletal_length = sk.calculate_skeleton_distance(curr_limb.skeleton)

    #4) Return n_spine/length
    return total_spines/skeletal_length

@run_options(run_type="Branch")
def axon_branch(curr_branch,name=None,branch_name=None,**kwargs):
    if "axon" in curr_branch.labels or "Axon" in curr_branch.labels:
        return True
    else:
        return False

In [ ]:
import neuron_searching as ns

functions_list=[
limb_n_spine,
limb_n_spine_per_skeletal_length,
"width",
axon_branch

]

returned_output = ns.query_neuron(inhibitory_obj,
                         functions_list,
                          query="limb_n_spine > 1",
                          return_dataframe=True,
                          return_limbs=False,
                          return_limb_grouped_branches=True,
                         print_flag=False)
returned_output

In [ ]:
returned_output = ns.query_neuron(inhibitory_obj,
                         functions_list,
                          query="limb_n_spine > 8 or axon_branch == True",
                          return_dataframe=False,
                          return_limbs=False,
                          return_limb_grouped_branches=True,
                         print_flag=False)
returned_output

# How to identify an axon

In [ ]:
returned_output = ns.query_neuron(inhibitory_obj,
                         functions_list,
                          query="limb_n_spine < 6 and width < 400",
                          return_dataframe=False,
                          return_limbs=False,
                          return_limb_grouped_branches=True,
                         print_flag=False)
returned_output

# Demo Max Analysis

In [ ]:
soma_obj = inhibitory_obj.concept_network.nodes["S0"]["data"]
soma_obj.mesh_center

In [ ]:
"""
Pseudocode: 
1) Get the soma mesh center
2) For every axon branch:
a. GEt the branch mesh center
b. Find the y distance above or below the soma center
c. RELU

3) Sum up results

***** THEN STORE IN A DATJOINT TABLE *****


"""
soma_obj = inhibitory_obj.concept_network.nodes["S0"]["data"]
soma_obj_mesh_center = soma_obj.mesh_center

total_above_soma_distance = 0

for limb_idx,branch_list in returned_output.items():
    for b in branch_list:
        print(f"---- Working on limb {limb_idx} branch {b} ------")
        branch_obj = inhibitory_obj.concept_network.nodes[limb_idx]["data"].concept_network.nodes[b]["data"]
        branch_obj_mesh_center = branch_obj.mesh_center
        
        y_difference = branch_obj_mesh_center[1] - soma_obj_mesh_center[1]
        print(f"y_difference = {y_difference}")
        
        #c. RELU
        if y_difference < 0:
            y_difference = 0
        
        print(f"y_difference = {y_difference}")
        
        total_above_soma_distance += y_difference
    
total_above_soma_distance

# visualize results

In [ ]:
nviz.visualize_neuron(inhibitory_obj,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict=returned_output,
                      mesh_color="red",
                     mesh_whole_neuron=True,
                     axis_box_off=False)